In [46]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from common import *
import qgrid

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Exchange

In [49]:
# Public informaiton
exchanges = [c.BINANCE, c.GDAX, c.POLONIEX]
a = Asset(c.ETH, c.BTC)
for ex in exchanges:
    print("Exchange", ex)
    exchange = load_exchange(ex)
    print(exchange.timeframes)
    exchange.get_markets()
    exchange.fetch_ohlcv(a.symbol, '5m')
    exchange.fetch_order_book(a.symbol)
    exchange.fetch_public_trades(a.symbol)
    exchange.fetch_ticker(a.symbol)

Exchange binance
{'1m': '1m', '3m': '3m', '5m': '5m', '15m': '15m', '30m': '30m', '1h': '1h', '2h': '2h', '4h': '4h', '6h': '6h', '8h': '8h', '12h': '12h', '1d': '1d', '3d': '3d', '1w': '1w', '1M': '1M'}
Exchange gdax
{'1m': 60, '5m': 300, '15m': 900, '30m': 1800, '1h': 3600, '2h': 7200, '4h': 14400, '12h': 43200, '1d': 86400, '1w': 604800, '1M': 2592000, '1y': 31536000}
Exchange poloniex
{'5m': 300, '15m': 900, '30m': 1800, '2h': 7200, '4h': 14400, '1d': 86400}


In [50]:
# Account Information
exchanges = [c.BINANCE, c.GDAX]
a = Asset(c.ETH, c.BTC)
for ex in exchanges:
    print("Exchange", ex)
    exchange = load_exchange(ex)
    exchange.fetch_balance()
    exchange.fetch_orders(a.symbol)
    exchange.fetch_open_orders(a.symbol)
    exchange.fetch_closed_orders(a.symbol)

Exchange binance
Exchange gdax


In [51]:
exchanges = [c.BINANCE]
a = Asset(c.BTS, c.BTC)

# Market BUY
for ex in exchanges:
    print("Exchange", ex)
    exchange = load_exchange(ex)
    balance = exchange.fetch_balance()
    print(a.base, balance[a.base], a.quote, balance[a.quote])
    order = exchange.create_market_buy_order(a.symbol, 1)
    print(order)

Exchange binance
BTS {'free': 0.955, 'used': 0.0, 'total': 0.955} BTC {'free': 0.05005258, 'used': 0.0, 'total': 0.05005258}
{'info': {'symbol': 'BTSBTC', 'orderId': 3511379, 'clientOrderId': 'Ydx2Ooh2L1et7e2eep1LeI', 'transactTime': 1515289809612, 'price': '0.00000000', 'origQty': '1.00000000', 'executedQty': '1.00000000', 'status': 'FILLED', 'timeInForce': 'GTC', 'type': 'MARKET', 'side': 'BUY'}, 'id': '3511379'}


In [52]:
exchanges = [c.BINANCE]
a = Asset(c.BTS, c.BTC)

# Market SELL
for ex in exchanges:
    print("Exchange", ex)
    exchange = load_exchange(ex)
    balance = exchange.fetch_balance()
    print(a.base, balance[a.base], a.quote, balance[a.quote])
    found_order = exchange.fetch_order(order['info']['orderId'], a.symbol)
    print("Placed order", found_order)    

Exchange binance
BTS {'free': 1.954, 'used': 0.0, 'total': 1.954} BTC {'free': 0.05000476, 'used': 0.0, 'total': 0.05000476}
Placed order {'info': {'symbol': 'BTSBTC', 'orderId': 3511379, 'clientOrderId': 'Ydx2Ooh2L1et7e2eep1LeI', 'price': '0.00000000', 'origQty': '1.00000000', 'executedQty': '1.00000000', 'status': 'FILLED', 'timeInForce': 'GTC', 'type': 'MARKET', 'side': 'BUY', 'stopPrice': '0.00000000', 'icebergQty': '0.00000000', 'time': 1515289809612, 'isWorking': True}, 'id': '3511379', 'timestamp': 1515289809612, 'datetime': '2018-01-07T01:50:10.612Z', 'symbol': 'BTS/BTC', 'type': 'market', 'side': 'buy', 'price': 0.0, 'amount': 1.0, 'cost': 0.0, 'filled': 1.0, 'remaining': 0.0, 'status': 'closed', 'fee': None}


In [ ]:
exchanges = [c.BINANCE]
a = Asset(c.BTS, c.BTC)

# Sell all remaining quantity
for ex in exchanges:
    # For very small amounts the exchanges will deny the request
    # We need to handle this in the order_manager or executor
    print("Exchange", ex)
    exchange = load_exchange(ex)
    balance = exchange.fetch_balance()[a.base]
    print(a.base, balance['total'])
    order = exchange.create_market_sell_order(a.symbol, balance['total'])
    print(order)

In [ ]:
exchange.fetch_balance()[a.base]

### Ingest Data

In [39]:
base = c.LTC
quote = c.USDT
exchange = load_exchange(c.BINANCE)
asset = Asset(base, quote)
assets = [ Asset(coin, quote) for coin in [c.BTC, c.ETH, c.LTC] ]
period = '1m' 
start = datetime.datetime.utcnow() - datetime.timedelta(hours=2)
end = datetime.datetime.utcnow() - datetime.timedelta(hours=0)

In [41]:
# Single Coin
df = ohlcv.fetch_and_save_ohlcv_data(exchange, asset, period, start, end)
df.head()

Downloading: LTC/USDT
Downloaded rows: 120


AttributeError: 'CCXTExchange' object has no attribute 'id'

In [43]:
exchange.client.id

'binance'

In [ ]:
# Load from File
fpath = ohlcv.get_price_data_fpath(asset, exchange.id, period)
df = ohlcv.load_chart_data_from_file(fpath)

In [ ]:
# Multiple Coins
ohlcv.download_chart_data(exchange, assets, period, start, end)
df = ohlcv.load_multiple_assets(exchange.id, assets, period, start)
df.head()

### Feeds

In [ ]:
# CSV Feed
feed_fpath = ohlcv.get_price_data_fpath(asset, exchange.id, period='1m')
feed = CSVDataFeed(feed_fpath)
feed.initialize()

In [ ]:
# Grab 1 row at a time
for i in range(3):
    data = feed.next()
    print(data['time_utc'], data['close'])

In [ ]:
# Access all rows in history
feed.history().head()

In [ ]:
# Exchange Live Feed
start = datetime.datetime.utcnow() - datetime.timedelta(hours=1)
end = None
feed_fpath = ohlcv.get_price_data_fpath(asset, exchange.id, period='1m')

In [ ]:
feed = ExchangeDataFeed(exchange, [assets[0]], period, feed_fpath, start, end)
feed.initialize()

In [ ]:
# Access History
feed.history(t_minus=3).tail()

In [ ]:
# Grab 1 row at a time (don't refresh data)
for i in range(3):
    data = feed.next(refresh=False)
    print(data['time_utc'], data['close'])

In [ ]:
# Refresh data before next query
data = feed.next(refresh=True)
print(data['time_utc'], data['close'])

In [ ]:
# Example Using the Feed

def my_strategy(row, history, exchange):
    print("Time:", row['time_utc'], "Price:", row['close'])
    if row['close'] > .024:
        print("Buying coin")
        #res = exchange.limit_buy(quantity=1, price=5)
    else:
        print("Selling that stupid coin")
    return {
        'time': row['time_utc'],
        'close': row['close'],
        'action': 'buy',
        'quantity': 1,
        'price': 5,
        'exchange': exchange.id
    }

def backtest(feed, strategy, exchange):
    results = []
    row = feed.next()
    while row is not None:
        output = strategy(row, feed, exchange)
        row = feed.next(refresh=True)
        results.append(output)
    return results

In [ ]:
res = backtest(feed, my_strategy, exchange)

### Order

In [ ]:
asset = Asset(c.LTC, c.USDT)
order = Order(
    exchange_id=exchange.id, 
    asset=asset,
    price=250., 
    quantity=1, 
    order_type=OrderType.LIMIT_BUY
)
order

### Strategy

In [ ]:
# https://www.backtrader.com/docu/quickstart/quickstart.html
# https://enigmampc.github.io/catalyst/beginner-tutorial.html#basics

class Strategy():
    def __init__(self, params):
        self.params = params
        self.initialize()
    
    def initialize(self):
        self.ctx.symbol = 'BTC/USD'
    
    def handle_data(self, data):
        # Handle next iteration of data from feed
        pass
    
class TestStrategy():
    def __init__(self):
        pass
    
    def next(self, data):
        pass

### Plotting

In [ ]:
utils.charts.plot_range(feed.history(), start, end, 'close')

### Exchange

In [ ]:
ex = TestExchange()
print(ex.fetch_balance())
orders = ex.limit_buy(c.BTC, c.USD, 1, 15000.0)
orders

In [ ]:
EX = CCXTExchange(gdax)

### Runner

In [ ]:
@unique
class TradeMode(Enum):
    BACKTEST = 0
    SIMULATE = 1
    LIVE = 2


class Punisher():
    def __init__(self, exchange, feed, strategy, record):
        self.exchange = exchange
        self.feed = feed
        self.strategy = strategy
        self.record = record
        
    def backtest(self):
        results = []
        row = self.feed.next()
        while row is not None:
            print("Timestep", row['time_utc'], "Price", row['close'])
            row = self.feed.next()
            output = self.strategy(row, self.exchange, self.feed)
            results.append(output)
        return results
    
    def simulate(self):
        while True:
            row = self.feed.next()
            if row is not None:
                output = self.strategy(row, self.exchange, self.feed)
                self.record['test'].append(output)
            time.sleep(2)

    def live(self):
        while True:
            row = self.feed.next()
            if row is not None:
                output = self.strategy(row, self.exchange, self.feed)
                self.record['live'].append(output)
            time.sleep(2)
    
    def punish(self, mode=False):
        if mode == TradeMode.BACKTEST:
            print("Backtesting ...")
            self.backtest()
        elif mode == TradeMode.SIMULATE:
            print("Simulating orders ...")
            self.simulate()
        elif mode == TradeMode.LIVE:
            print("LIVE TRADING! CAREFUL!")
            self.live()